# MASTER CONTROLLER: CASCADE

In [1]:
# ALWAYS RUN THIS CELL
# widen jupyter notebook window
from IPython.display import display, HTML
display(HTML("<style>.container {width:95% !important; }</style>"))

In [2]:
import os
from pathlib import Path

import paramiko

import getpass
import gc
import time

In [3]:
# dir_github = '/media/rich/Home_Linux_partition/github_repos/'
# import sys
# sys.path.append(dir_github)

# # %load_ext autoreload
# # %autoreload 2
# # from s2p_on_o2 import util

In [4]:
# paramiko.util.log_to_file("paramiko.log")

In [5]:
from bnpm import server

----
# ==== Optional ====
### transfer raw data from local machine to `MICROSCOPE`

In [19]:
## Define directories for where the local data is and the remote directory where the contents of that folder should be copied to (MICROSCOPE)
dir_data_local = '/media/rich/bigSSD/tmp_data/'
dir_data_MICROSCOPE = '/n/files/Neurobio/MICROSCOPE/Rich/data/test'  ## contents of dir_data_local will be dumped into dir_data_MICROSCOPE

print(f'contents of    {dir_data_local}    will be copied to    {dir_data_MICROSCOPE}')

contents of    /media/rich/bigSSD/tmp_data/    will be copied to    /n/files/Neurobio/MICROSCOPE/Rich/data/test


In [8]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')
pw = server.pw_encode(getpass.getpass(prompt='Password: '))

Username: rh183
Password: ········


## Prepare `sftp` and `ssh` objects

In [11]:
sftp = server.sftp_interface(hostname="transfer.rc.hms.harvard.edu", port=22)
sftp.connect(username=username, password=server.pw_decode(pw))

In [ ]:
# sftp.mkdir_safe(dir_data_remote)
sftp.put_dir(dir_data_local, dir_data_MICROSCOPE)

------------
# ==== Start ====

## Prepare username + password

In [7]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')
pw = server.pw_encode(getpass.getpass(prompt='Password: '))

Username: rh183
Password: ········


## Prepare `sftp` and `ssh` objects

In [9]:
sftp = server.sftp_interface(hostname="transfer.rc.hms.harvard.edu", port=22)
sftp.connect(username=username, password=server.pw_decode(pw))

In [12]:
## initialize ssh_transfer
ssh_t = server.ssh_interface(
    nbytes_toReceive=20000,
    recv_timeout=1,
    verbose=True,
)
ssh_t.o2_connect(
    hostname=remote_host_transfer,
    username=username,
    password=server.pw_decode(pw),
    passcode_method=1,
    verbose=1,
    skip_passcode=False,    
)

Oops, unhandled type 3 ('unimplemented')


Duo two-factor login for rh183

Enter a passcode or select one of the following options:

 1. Duo Push to XXX-XXX-4151
 2. Phone call to XXX-XXX-4151
 3. SMS passcodes to XXX-XXX-4151

Passcode or option (1-3): 
1



Pushed a login request to your device...

Success. Logging you in...
Welcome to O2 (Orchestra 2)!

   ___    ____
  / _ \  |___ \
 | | | |   __) |
 | |_| |  / __/
  \___/  |_____|


You've landed on transfer05 which is a
4 core system with 7.62 GiB memory
running kernel 3.10.0 born on 2020-07-28

==== O2 ===================================================================

  News (Sep 8 2022)
  

  +-------------------------------------------------------------------------------+
  | ###################### Try out the new O2 Portal! ########################### |
  |                                                                               |
  | * Access O2 via the web! Just point your web browser to                       |
  |   o2portal.rc.hms.harvard.edu and login with

In [13]:
## initialize ssh_compute
ssh_c = server.ssh_interface(
    nbytes_toReceive=20000,
    recv_timeout=1,
    verbose=True,
)
ssh_c.o2_connect(
    hostname=remote_host_compute,
    username=username,
    password=server.pw_decode(pw),
    passcode_method=1,
    verbose=1,
    skip_passcode=False,    
)

Duo two-factor login for rh183

Enter a passcode or select one of the following options:

 1. Duo Push to XXX-XXX-4151
 2. Phone call to XXX-XXX-4151
 3. SMS passcodes to XXX-XXX-4151

Passcode or option (1-3): 
1



Pushed a login request to your device...

Success. Logging you in...
Welcome to O2 (Orchestra 2)!

   ___    ____
  / _ \  |___ \
 | | | |   __) |
 | |_| |  / __/
  \___/  |_____|


You've landed on login06 which is a
8 core system with 15.49 GiB memory
running kernel 3.10.0 born on 2020-07-07

==== O2 ===================================================================

  News (Sep 8 2022)
  

  +-------------------------------------------------------------------------------+
  | ###################### Try out the new O2 Portal! ########################### |
  |                                                                               |
  | * Access O2 via the web! Just point your web browser to                       |
  |   o2portal.rc.hms.harvard.edu and login with y

## Pull/update repo

In [14]:
ssh_c.send('cd /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP')
ssh_c.send('git pull')

ssh_c.expect(str_success=f'[{username}');

AP

git pull
(base) [rh183@login06 NBAP]$ git pull



## Activate environment

In [15]:
ssh_c.send('source activate Cascade')
ssh_c.expect(str_success=f'[{username}');

remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 36 (delta 12), reused 16 (delta 12), pack-reused 18
Unpacking objects: 100% (36/36), done.
From https://github.com/RichieHakim/NBAP
   f2476a4..a3f9d66  main       -> origin/main
Updating f2476a4..a3f9d66
Fast-forward
 .../analysis_BMI_rewards-checkpoint.ipynb          |  1796 ++
 .../master_controller-checkpoint.ipynb             |   107 +-
 .../CASCADE/master_controller.ipynb                |   107 +-
 ...is_BMI_activityVectorAlignment-checkpoint.ipynb | 19149 ++++++++++++++++++
 pipeline_res2p_BMI/CASCADE/dispatcher.py           |     5 +-
 pipeline_res2p_BMI/CASCADE/master_controller.ipynb |   151 +-
 .../analysis_BMI_activityVectorAlignment.ipynb     | 19156 +++++++++++++++++++
 pipeline_res2p_BMI/analysis_BMI_rewards.ipynb      |  1796 ++
 8 files changed, 42143 insertions(+), 124 deletions(-)
 create mode 100644 .ipynb_checkpoints/anal

## Prepare directories

In [194]:
import numpy as np

mouse_name = 'mouse_1511L'
date = '20230112'

name_slurm = mouse_name + '_' + date[-2:]

# dir_motionData_MICROSCOPE = (Path('/n/files/Neurobio/MICROSCOPE/Ally/Mesoscope') / mouse_name / date).as_posix()
# dir_s2p_MICROSCOPE = (Path('/n/files/Neurobio/MICROSCOPE/Ally/Mesoscope') / mouse_name / date / 'suite2p_o2_output/jobNum_0/suite2p').as_posix()
dir_s2p = str((Path('/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/') / mouse_name / date / 'scanimage_data' / 'exp' / 'jobNum_0' / 'suite2p' / 'plane0'))
# dir_s2p = str((Path('/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/') / mouse_name / date / 'scanimage_data' / 'exp' / 'plane0'))

# dir_data_remote = (Path('/n/data1/hms/neurobio/sabatini/rich/data/2pRAM') / mouse_name / date).as_posix()

dir_saveOutputs = (Path('/n/data1/hms/neurobio/sabatini/rich/analysis/spike_inference') / mouse_name / date).as_posix()

# fileList = sftp.sftp.listdir(dir_motionData)
# fileName = sorted(np.array(fileList)[np.array([('Motion' in file)*('.csv' in file) for file in fileList], dtype=np.bool8)])[-1]
# path_motionData_MICROSCOPE = str(Path(dir_motionData) / fileName)

name_job = 'jobNum_'


# path_motionData_MICROSCOPE = (Path(dir_motionData_MICROSCOPE) / fileName).as_posix()

# path_dispatcher_local = Path(r'C:\Users\scanimage\github_repos\s2p_on_o2').resolve() / 'dispatcher.py'  ## path to the dispatcher.py file on local computer
path_dispatcher_local = Path(r'/media/rich/Home_Linux_partition/github_repos/NBAP/pipeline_res2p_BMI/CASCADE/').resolve() / 'dispatcher.py'  ## path to the dispatcher.py file on local computer
path_dispatcher_remote = (Path(dir_saveOutputs) / 'dispatcher.py').as_posix()  ## path to where you want the dispatcher.py file to be copied onto the server
path_script_remote = Path(r'/n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/pipeline_res2p_BMI/CASCADE/run_cascade.py').as_posix()  ## path to a copy of the remote_run_s2p.py file within the s2p_on_o2 repo on the server

# print(f'contents of    {dir_motionData_MICROSCOPE}   and   {dir_s2p_MICROSCOPE}    will be copied to    {dir_data_remote}')

## Prepare commands

In [195]:
## Expectation for dispatch args: 
# path_selfScript = args[0] = path_dispatcher_remote
# dir_save = args[1] = dir_S2pOutput_remote
# path_script = args[2] = path_s2pScript_remote
# name_job = args[3] = name_job
# dir_fastDisk = args[4] = dir_fastDisk_remote
# name_slurm = args[5] = name_slurm
# dir_data = args[6] = dir_data_remote


prompt_snip = f'[{username}'

commands = {
#     'make_dir': f"mkdir -p {dir_data_remote}",
#     'copy_motionData': f"cp -r {path_motionData_MICROSCOPE} {dir_data_remote}",
#     'copy_s2p': f"cp -r {dir_s2p_MICROSCOPE} {dir_data_remote}",
    'dispatch_s2p': f"python {str(path_dispatcher_remote)} {dir_saveOutputs} {path_script_remote} {name_job} {name_slurm} {dir_s2p}"  ## dispatcher expecting these args as inputs
}
display(commands)

{'dispatch_s2p': 'python /n/data1/hms/neurobio/sabatini/rich/analysis/spike_inference/mouse_1511L/20230112/dispatcher.py /n/data1/hms/neurobio/sabatini/rich/analysis/spike_inference/mouse_1511L/20230112 /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/pipeline_res2p_BMI/CASCADE/run_cascade.py jobNum_ mouse_1511L_12 /n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_1511L/20230112/scanimage_data/exp/jobNum_0/suite2p/plane0'}

## Upload `dispatcher.py` file

In [196]:
sftp.mkdir_p(Path(path_dispatcher_remote).parent.as_posix())
sftp.sftp.put(str(path_dispatcher_local), str(path_dispatcher_remote));

## Copy data from `MICROSCOPE` to `data1`

## Dispatch `remote_run_s2p.py`

In [197]:
commands['dispatch_s2p']

'python /n/data1/hms/neurobio/sabatini/rich/analysis/spike_inference/mouse_1511L/20230112/dispatcher.py /n/data1/hms/neurobio/sabatini/rich/analysis/spike_inference/mouse_1511L/20230112 /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/pipeline_res2p_BMI/CASCADE/run_cascade.py jobNum_ mouse_1511L_12 /n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_1511L/20230112/scanimage_data/exp/jobNum_0/suite2p/plane0'

In [198]:
ssh_c.send(commands['dispatch_s2p']);

ssh_c.expect(str_success=prompt_snip);

_1511L/20230112/scanimage_data/exp/jobNum_0/suite2p/plane0s/suite2p_output/mouse 

/n/data1/hms/neurobio/sabatini/rich/analysis/spike_inference/mouse_1511L/20230112/dispatcher.py /n/data1/hms/neurobio/sabatini/rich/analysis/spike_inference/mouse_1511L/20230112 /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/pipeline_res2p_BMI/CASCADE/run_cascade.py jobNum_ /n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_1511L/20230112/scanimage_data/exp/jobNum_0/suite2p/plane0
Submitting job: jobNum_ 0

Submitted batch job 1847282

(Cascade) [rh183@login06 NBAP]$ 


### check on job

In [203]:
ssh_c.send(cmd=f'squeue -u {username} --Format=jobid,name,partition,state,timeused,starttime,timelimit,reasonlist,minmemory,mincpus,gres');
ssh_c.expect(str_success=f"[{username}", total_timeout=10);

ate,timeused,starttime,timelimit,reasonlist,minmemory,mincpus,gres
JOBID               NAME                PARTITION           STATE               TIME                START_TIME          TIME_LIMIT          NODELIST(REASON)    MIN_MEMORY          MIN_CPUS            TRES_PER_NODE       
1847282             mouse_1511L_12      short               RUNNING             15:34               2023-01-23T03:40:07 4:00:00             compute-a-17-121    32G                 20                  N/A                 
(Cascade) [rh183@login06 NBAP]$ 


In [208]:
## I think this code block caused an out of memory error on a few of my jobs... not sure why

# ssh_c.send(cmd=f"cat {(Path(dir_S2pOutput_remote) / (name_job+'0') / 'print*').as_posix()}")
# ssh_c.expect(str_success=f"[{username}", total_timeout=1);

In [ ]:
ssh_c.send(cmd=f"cat {(Path(dir_S2pOutput_remote) / (name_job+'0') / 'print*').as_posix()}")
ssh_c.expect(str_success=f"RUN COMPLETE", recv_timeout=0.3, total_timeout=60*60*10, sleep_time=0.1, verbose=False);

print(f'RUN COMPLETE!!!     {time.ctime()}')

## Clean up

In [ ]:
## delete password
del pw
gc.collect()

## close sftp
if sftp: sftp.close()
if transport: transport.close()
del sftp
del transport
    
## close ssh
ssh_t.close()
del ssh_t
ssh_c.close()
del ssh_c
gc.collect()

----
# ==== Optional ====
### transfer s2p outputs back from `data1` to `MICROSCOPE`

## Prepare username + password

In [5]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')
pw = util.pw_encode(getpass.getpass(prompt='Password: '))

Username: rh183
Password: ········


## Prepare `ssh` object

In [ ]:
## initialize ssh_transfer
ssh_t = util.ssh_interface(
    nbytes_toReceive=4096,
    recv_timeout=1,
    verbose=True,
)
ssh_t.o2_connect(
    hostname=remote_host_transfer,
    username=username,
    password=util.pw_decode(pw),
    passcode_method=1,
    verbose=1,
    skip_passcode=False,
)

In [274]:
dir_S2pOutput_remote = dir_S2pOutput_remote  ## from above
dir_s2pOutput_MICROSCOPE = (Path(dir_data_MICROSCOPE) / 'suite2p_o2_output').as_posix()  ## from above
print(f'contents of    {dir_S2pOutput_remote}    will be copied to    {dir_s2pOutput_MICROSCOPE}')

contents of    /n/data1/hms/neurobio/sabatini/rich/analysis/suite2p_output/AEG21/2022_05_14    will be copied to    /n/files/Neurobio/MICROSCOPE/Ally/Mesoscope/AEG21/2022_05_14/suite2p_o2_output


In [275]:
commands = {
    'make_dir': f"mkdir -p {dir_s2pOutput_MICROSCOPE}",
    'copy_s2p': f"cp -r {(Path(dir_S2pOutput_remote)).as_posix()}/. {dir_s2pOutput_MICROSCOPE}",
}
display(commands)

{'make_dir': 'mkdir -p /n/files/Neurobio/MICROSCOPE/Ally/Mesoscope/AEG21/2022_05_14/suite2p_o2_output',
 'copy_s2p': 'cp -r /n/data1/hms/neurobio/sabatini/rich/analysis/suite2p_output/AEG21/2022_05_14/. /n/files/Neurobio/MICROSCOPE/Ally/Mesoscope/AEG21/2022_05_14/suite2p_o2_output'}

In [276]:
ssh_t.send_receive(commands['make_dir']);

e/AEG21/2022_05_14/suite2p_o2_output



In [277]:
ssh_t.send(commands['copy_s2p'])
ssh_t.expect(str_success=f'[{username}', total_timeout=120, verbose=True);

(base) [rh183@transfer03 ~]$ 


## Clean up

In [ ]:
## delete password
del pw
gc.collect()
    
## close ssh
ssh_t.close()
del ssh_t
gc.collect()

----
# ==== Optional ====
### transfer s2p outputs from `MICROSCOPE` to local machine

In [39]:
dir_s2pOutput_MICROSCOPE = dir_s2pOutput_MICROSCOPE  ## from above
dir_s2pOutput_local = str(Path(dir_data_local).resolve() / 'suite2p_o2_output')
print(f'contents of    {dir_s2pOutput_MICROSCOPE}    will be copied to    {dir_s2pOutput_local}')

contents of    /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output    will be copied to    /media/rich/bigSSD/tmp_data/suite2p_o2_output


In [57]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')
pw = util.pw_encode(getpass.getpass(prompt='Password: '))

Username: rh183
Password: ········


## Prepare `sftp` and `ssh` objects

In [40]:
sftp = util.sftp_interface(hostname="transfer.rc.hms.harvard.edu", port=22)
sftp.connect(username=username, password=util.pw_decode(pw))

In [41]:
sftp.get_dir(dir_s2pOutput_MICROSCOPE, dir_s2pOutput_local)

downloading /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output/jobNum_0/params.json   to   /media/rich/bigSSD/tmp_data/suite2p_o2_output/jobNum_0/params.json
downloading /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output/jobNum_0/print_log_53419173.log   to   /media/rich/bigSSD/tmp_data/suite2p_o2_output/jobNum_0/print_log_53419173.log
downloading /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output/jobNum_0/batch_run_output.png   to   /media/rich/bigSSD/tmp_data/suite2p_o2_output/jobNum_0/batch_run_output.png
downloading /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output/jobNum_0/log.txt   to   /media/rich/bigSSD/tmp_data/suite2p_o2_output/jobNum_0/log.txt
downloading /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output/jobNum_0/sbatch_config.sh   to   /media/rich/bigSSD/tmp_data/suite2p_o2_output/jobNum_0/sbatch_config.sh
downloading /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output/jobNum_0/remote_run_s2p.py   to   /medi

## Clean up

In [43]:
## delete password
del pw
gc.collect()

## close sftp
sftp.close()
del sftp

In [42]:
ssh_t.close()
ssh_c.close()